In [1]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
import sklearn.metrics
import  numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
File_object = open(r"cran.all", "r")

In [ ]:
abstracts = File_object.read().split(".I ")[1:]

In [ ]:
abstracts

In [ ]:
raw_corpus = [" ".join(abstract.splitlines()) for abstract in abstracts]

In [ ]:
tokenized_corpus = [re.sub(' +', ' ', re.sub(r'[^\w\s]', ' ', doc[doc.find('.W') + 2:doc.find('Etc')]))[:-1] for doc in raw_corpus]

In [ ]:
tokenized_corpus

In [ ]:
File_object2 = open(r"query.text", "r")

In [ ]:
queries = File_object2.read().split(".I ")[1:]

In [ ]:
queries

In [ ]:
raw_queries = [" ".join(query.splitlines()) for query in queries]

In [ ]:
tokenized_queries =  [re.sub(' +', ' ', re.sub(r'[^\w\s]', ' ', query[query.find('.W') + 2:query.find('Etc')]))[:-1] for query in raw_queries]

In [ ]:
tokenized_queries

In [ ]:
tdif_vectorizer = TfidfVectorizer(lowercase=True, stop_words=text.ENGLISH_STOP_WORDS)
corpus_tdif = tdif_vectorizer.fit_transform(tokenized_corpus)
query_tdif = tdif_vectorizer.transform(tokenized_queries)
print(corpus_tdif.toarray().shape)
print(query_tdif.toarray().shape)

In [ ]:
bi_vectorizer = CountVectorizer(lowercase=True, stop_words=text.ENGLISH_STOP_WORDS, binary=True)
corpus_bi = bi_vectorizer.fit_transform(tokenized_corpus)
query_bi = bi_vectorizer.transform(tokenized_queries)
print(corpus_bi.toarray().shape)
print(query_bi.toarray().shape)

In [ ]:
def getTopEuclidean(corpus, queries):
    euclideanMatrix = sklearn.metrics.pairwise_distances(X=queries, Y=corpus, metric='euclidean')
    minValues = []
    for query in euclideanMatrix:
        minValues.append(np.argsort(query)[:10])
    return np.array(minValues)

In [ ]:
def getTopCosineSimilarity(corpus, queries):
    cosineSimilarityMatrix = sklearn.metrics.pairwise.cosine_similarity(X=queries, Y=corpus, dense_output=True)
    minValues = []
    for query in cosineSimilarityMatrix:
        minValues.append(np.argsort(query)[:10])
    return np.array(minValues)

In [ ]:
eucclidian_tdif = getTopEuclidean(corpus_tdif.toarray(), query_tdif.toarray())

In [ ]:
cosineSimilarity_tdif = getTopCosineSimilarity(corpus_tdif.toarray(), query_tdif.toarray())

In [ ]:
relDocs = np.loadtxt("qrels.text")
relDocs = np.delete(relDocs, 2, 1)
relDocs = np.delete(relDocs, 2, 1)
relDocs[:29]

In [ ]:
relDocsMatrix = []
index = 0
relCount = relDocs.shape[0]
for query in range(1, 226):
    queryDocs = []
    while index < relCount and relDocs[index][0] == query:
        queryDocs.append(relDocs[index][1])
        index += 1
    relDocsMatrix.append(queryDocs)  
relDocsMatrix[0]

In [ ]:
def precision(actual, relavent):
    return np.in1d(actual, relevant).sum() / actual.shape[0]

In [ ]:
def recall(actual, relevant):
    return np.in1d(actual, relevant).sum() / relevant.shape[0]

In [ ]:
def fscore(precision, recall):
    (precision + recall) / 2

In [3]:
def calcScores(actualMat, relevantMat):
    pData = []
    rData = []
    fData = []
    for query in range(225):
        p = precision(actualMat[query], np.array(relevantMat[query]))
        r = recall(actualMat[query], np.array(relevantMat[query]))
        f = fscore(p, r)
        pData.append(p)
        rData.append(r)
        fData.append(f)
    return pData, rData, fData

In [2]:
def makeBarGraph(data, title, legend, yLabel):
    plt.bar([i for i in range(1, 226)], data, color='red')
    plt.xticks(ticks=[0, 50, 100, 150, 200], labels=["0", "50", "100", "150", "200"])
    plt.yticks(ticks=[0.0, 0.2, 0.4, 0.6, 0.8], labels=["0.0", "0.2", "0.4", "0.6", "0.8"])
    plt.xlabel("Query Index")
    plt.ylabel(yLabel)
    plt.legend([legend], loc='upper right')
    plt.title(title)
    plt.show()